In [60]:
from pickle import load as pickle_load
from os import path as os_path
from string import punctuation as string_punctuation
from re import search as regex_search
from re import split as regex_split
from re import sub as regex_substitute
from heapq import heapify, heappop, heappush

In [38]:
## this algorithm is taken from https://www.geeksforgeeks.org/python-program-for-binary-search/

# Iterative Binary Search Function
# It returns index of search_element in given dictionary arr if present,
# else returns -1
def binarySearch(dictionary, search_element):
    low = 0
    high = len(dictionary) - 1
    mid = 0
 
    while low <= high:
 
        mid = (high + low) // 2
 
        # If search_element is greater, ignore left half
        if dictionary[mid] < search_element:
            low = mid + 1
 
        # If search_element is smaller, ignore right half
        elif dictionary[mid] > search_element:
            high = mid - 1
 
        # means search_element is present at mid
        else:
            return mid
 
    # If we reach here, then the element was not present
    return -1

In [18]:
def getClitics():

    clitics = set()
    clitic_path = os_path.join('.', 'clitics.txt')

    with open(clitic_path) as stop_file:
        stop_lines = stop_file.readlines()

    for line in stop_lines:
        stopword = line[:-1]
        clitics.add(stopword)

    return clitics

In [25]:
with open('dictionary.pickle', 'rb') as dictionary_file:
    dictionary = pickle_load(dictionary_file)

dictionary_file.close()


with open('index.pickle', 'rb') as index_file:
    inverted_index = pickle_load(index_file)

index_file.close()


with open('stopwords.pickle', 'rb') as stopword_file:
    stopwords = pickle_load(stopword_file)

stopword_file.close()

clitics = getClitics()

In [28]:
def normalize(token_list, clitics_set):

    # regular expression that will detect either a word, words with hyphens included, or a token composing of numbers
    # with special characters('/', ':', '.', ',') like 16/20, 19.02.2021, etc.
    shave_string = f'[{string_punctuation}]*([\d/:.,]+\d+)|([\w-]+)[{string_punctuation}]*'

    # a sentence will probably end with one of the following punctuation marks: '.', '!', '?', '...'
    end_of_sentence = r'[.?!]|[...]'
    # a flag that will show that the previous word is the last word of the sentence, which means the current word is the beginning of the sentence
    sentence_beginning = False

    # final document as a list of its tokens
    final_document = []

    tokens_after_normalization = 0

    for word in token_list:
            
        if word.lower() not in clitics_set:

            # shave the token
            shaved = regex_search(shave_string, word)
            if shaved is not None:
                # as described in the definition of shave_string, token can be in one of the 2 different forms
                token = shaved.group(1) if shaved.group(1) is not None else shaved.group(2)

                # # keep terms before the normalization
                # dictionary_before_normalization.add(token)

                # detect hyphenated words
                dash_search = regex_search('-', token)

                # after this conditional block, we will have a list of tokens named as token_splitted
                if dash_search:
                    # if the first character of the hyphenated word is upper, like Hewett-Pickard or New York-San Fransisco, then they should be 
                    # splitted and taken as different strings. if the first character is lower, then all hyphens should be deleted and the result will be
                    # one word('know-how' to 'knowhow')
                    tokens_splitted = regex_split('-', token) if token[0].isupper() else [regex_substitute('-', '', token)]
                else:
                    # if no hyphen detected, then the token is taken directly
                    tokens_splitted = [token]

                for splitted in tokens_splitted:
                    if splitted:
                        # if the first character of the token is upper and it is not the beginning of the sentence, then keep the token with uppercase letters.
                        # otherwise, lower the letters
                        splitted = splitted if (splitted[0].isupper() and (not sentence_beginning)) else splitted.lower()
                        
                        # # keep terms after normalization
                        # dictionary_after_normalization.add(splitted)
                        # keep the number of tokens after normalization
                        tokens_after_normalization += 1

                        final_document.append(splitted)

            end_of_sentence_search = regex_search(end_of_sentence, word)
            sentence_beginning = end_of_sentence_search is not None
        else:
            # # keep terms before the normalization
            # dictionary_before_normalization.add(word)

            token = word.lower()

            # # keep terms after normalization
            # dictionary_after_normalization.add(word)
            # keep the number of tokens after normalization
            tokens_after_normalization += 1
            
            final_document.append(splitted)

    return final_document, tokens_after_normalization

In [42]:
def returnTopK(elements, k):
    
    max_heap = [(-sort_base, return_element) for return_element, sort_base in elements]
    heapify(max_heap)

    result = []

    counter = 0
    elements_length = len(elements)
    while counter < k and counter < elements_length:
        sort_base, return_element = heappop(max_heap)
        result.append((return_element, -sort_base))
        counter += 1

    return result

In [43]:
def intersect(first_list, second_list):
    result = []

    first_index = 0
    second_index = 0
    while first_index < len(first_list) and second_index < len(second_list):
        
        if first_list[first_index] == second_list[second_index]:
            result.append(first_list[first_index])
            first_index += 1
            second_index += 1
        
        elif first_list[first_index] < second_list[second_index]:
            first_index += 1

        else:
            second_index += 1

    return result

In [54]:
def mergeLists(document_id_lists):
    postings_length = [(index, len(document_id_lists[index])) for index in range(len(document_id_lists))]
    length_order = returnTopK(postings_length, len(postings_length))

    result_list = document_id_lists[length_order[0][0]]
    for i in range(1, len(length_order)):
        index_second = length_order[i][0]
        second_list = document_id_lists[index_second]

        result_list = intersect(result_list, second_list)

        if len(result_list) == 0:
            return []
        
    return result_list


In [63]:
def phraseQuery(elements):
    global clitics, inverted_index, dictionary

    normalized_query, _ = normalize(elements, clitics)

    document_id_lists = []
    dictionary_positions = []
    for term in normalized_query:
        position = binarySearch(dictionary, term)

        if position == -1:
            return []
        else:
            document_ids = list(inverted_index[position][1].keys())
            # document_ids.sort()
            document_id_lists.append(document_ids)

            dictionary_positions.append(position)

    return mergeLists(document_id_lists), dictionary_positions

In [64]:
def proximityQuery(first_word, second_word, k):
    global inverted_index
    
    lookup_lists, dictionary_indexes = phraseQuery([first_word, second_word])
    result = []

    for document_id in lookup_lists:
        positions_first = inverted_index[dictionary_indexes[0]][1][document_id]
        positions_second = inverted_index[dictionary_indexes[1]][1][document_id]

        index_first = 0
        index_second = 0
        while index_first < len(positions_first) and index_second < len(positions_second):
            
            first_position = positions_first[index_first]
            second_position = positions_second[index_second]
            distance = abs(first_position - second_position)

            if distance <= k:
                result.append(document_id)
                break
            
            elif first_position < second_position:
                index_first += 1
            else:
                index_second += 1

    return result

In [65]:
phraseQuery(['showers', 'continued'])
proximityQuery('total', 'crop', 2)

[1, 1299, 1623, 2436, 3981, 4898, 15043, 15686]